In [1]:
model_id = "EleutherAI/gpt-j-6B"
revision = "float16"  # use float16 weights to fit in 16GB GPUs
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

In [2]:
import ray

In [3]:
ray.init(
    "ray://example-cluster-kuberay-head-svc:10001",
    runtime_env={
        "pip": [
            "accelerate>=0.16.0",
            "transformers>=4.26.0",
            "numpy<1.24",  # remove when mlflow updates beyond 2.2
            "torch",
        ]
    }
)

Python version:,3.10.9
Ray version:,2.3.0
Dashboard:,http://10.8.3.5:8265


In [4]:
import ray.data
import pandas as pd

ds = ray.data.from_pandas(pd.DataFrame([prompt] * 10, columns=["prompt"]))

(raylet) bash: /home/ray/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)


In [5]:
class PredictCallable:
    def __init__(self, model_id: str, revision: str = None):
        from transformers import AutoModelForCausalLM, AutoTokenizer
        import torch

        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            revision=revision,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            device_map="auto",  # automatically makes use of all GPUs available to the Actor
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)

    def __call__(self, batch: pd.DataFrame) -> pd.DataFrame:
        tokenized = self.tokenizer(
            list(batch["prompt"]), return_tensors="pt"
        )
        input_ids = tokenized.input_ids.to(self.model.device)
        attention_mask = tokenized.attention_mask.to(self.model.device)

        gen_tokens = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            do_sample=True,
            temperature=0.9,
            max_length=100,
            pad_token_id=self.tokenizer.eos_token_id,
        )
        return pd.DataFrame(
            self.tokenizer.batch_decode(gen_tokens), columns=["responses"]
        )

In [6]:
preds = (
    ds
    .repartition(100)
    .map_batches(
        PredictCallable,
        batch_size=4,
        fn_constructor_kwargs=dict(model_id=model_id, revision=revision),
        compute="actors",
        num_gpus=1,
    )
)

In [7]:
preds.take_all()

2023-04-14 00:01:16,033	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[repartition] -> ActorPoolMapOperator[MapBatches(PredictCallable)]
(raylet, ip=10.8.4.5) bash: /home/ray/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
MapBatches(PredictCallable):   0%|          | 0/100 [00:00<?, ?it/s](raylet) bash: /home/ray/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
(raylet, ip=10.8.5.4) bash: /home/ray/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
(raylet, ip=10.8.5.4) bash: /home/ray/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
(raylet) bash: /home/ray/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)


RayTaskError(RuntimeError): [36mray::_MapWorker.submit()[39m (pid=938, ip=10.8.5.4)
  File "/tmp/ray/session_2023-04-13_14-42-58_538184_8/runtime_resources/pip/9fc91d65ff21920e7899b7316ff121f0058d7e36/virtualenv/lib/python3.10/site-packages/transformers/utils/import_utils.py", line 1146, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "/home/ray/anaconda3/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/tmp/ray/session_2023-04-